In [1]:
%run functions.ipynb

Ready: 2023-05-03 00:39:52


In [2]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()
spark

In [71]:
def plot_matrix_rubric_cities(cnt_graphics=100, show=False, size_x=6.89*4, size_y=4.25*4, font_size=16, img_w=17.5, img_h=10.8, blank=False, word_sample='files/report.docx'):
    folder='ten_city'
    shar_nums = list(range(1,16))*500
    time = str(datetime.datetime.now())[2:16].replace('-','').replace(' ','_').replace(':','')
    print(f'files/{folder}/{folder}_shars_{time}.docx')
    df_src = pd.read_parquet(f'files/ten_city/requests_for_shars')
    df_src = pd.read_parquet(f'files/ten_city/requests_for_shars')
    df_new = df_src[['rubricName','city']].drop_duplicates().set_index(['rubricName','city'])
    df_new['Запросы 2020'] = df_src[df_src['year_r'] == 2020].groupby(['rubricName', 'city']).sum('cnt_requests')[['cnt_requests']]
    df_new['Запросы 2021'] = df_src[df_src['year_r'] == 2021].groupby(['rubricName', 'city']).sum('cnt_requests')[['cnt_requests']]
    df_new['Запросы 2022'] = df_src[df_src['year_r'] == 2022].groupby(['rubricName', 'city']).sum('cnt_requests')[['cnt_requests']]
    df_new['Запросы 2023'] = df_src[df_src['year_r'] == 2023].groupby(['rubricName', 'city']).sum('cnt_requests')[['cnt_requests']]
    df_new = df_new.fillna(0).astype(int)
    df_new['Запросы, итого'] =  df_new['Запросы 2020'] + df_new['Запросы 2021'] + df_new['Запросы 2022'] + df_new['Запросы 2023']
    df_new = df_new.reset_index()
    df_new['city_rubric'] = df_new['city'] + '.' + df_new['rubricName']
    df_new = df_new.set_index('city_rubric').sort_values('Запросы, итого', ascending=False)
    cnt_shars = 12
    step = 1
    rubrics = dd.read_parquet(f"files/ten_city/df_tx_grouped_filtered_ten_city")['rubricName'].unique().compute()
    rubrics = [i for i in rubrics.to_dict().values()]
    df_list = []
    start_range = 1
    stop_range = cnt_shars*step+1
    
    for graphic_num in tqdm(range(cnt_graphics)):
        shar_num = shar_nums.pop()
        
        df = df_new.iloc[range(start_range, stop_range, step)]
        start_range = stop_range
        stop_range = (graphic_num+2)*cnt_shars*step+1
        
        df['Запросы, итого'] = (df['Запросы, итого'] - min(df['Запросы, итого'])) / (max(df['Запросы, итого']) - min(df['Запросы, итого']))*300
        requests_total = df_new['Запросы 2023'].sum()
        df['Рост/Падение, %'] = df['Запросы 2023'] / df['Запросы 2022']
        df['Доля рынка, %'] = df['Запросы 2023'] / requests_total
        x = (df['Доля рынка, %'])
        y = (df['Рост/Падение, %'])
        sizes = (df['Запросы, итого']/1000)
        x_avg = (x.min()+x.max())/2
        y_avg = (y.min()+y.max())/2
        xi, yi = df['Доля рынка, %'], df['Рост/Падение, %']
        df.loc[(xi > x_avg) & (yi > y_avg), 'group'] = 'Звёзды:'
        df.loc[(xi > x_avg)  & (yi <= y_avg), 'group'] = 'Коровы:'
        df.loc[(xi <= x_avg)  & (yi > y_avg), 'group'] = 'Дети:'
        df.loc[(xi <= x_avg)  & (yi <= y_avg), 'group'] = 'Собаки:'
        df = df.sort_values(['Доля рынка, %', 'Рост/Падение, %'], ascending=[False, False])

        plt.rcParams['figure.figsize'] = size_x, size_y
        plt.rcParams['figure.dpi'] = 2**8
        plt.rcParams['font.size'] = '18'
        fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
        fig, ax = plt.subplots()
        ax.scatter(x, y, sizes, c='#C0504D', alpha=0.7)

        n = 0.02
        m = 0.02
        ax.set_ylim(ax.get_ylim()[0] * (1 - n), ax.get_ylim()[1] * (1 + n))
        ax.set_xlim(ax.get_xlim()[0] * (1 - m), ax.get_xlim()[1] * (1 + m))
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        plt.tick_params(bottom = False, left=False)
        #plt.title(rubric, font=fpath, fontsize=40, fontweight=1000)

        for xi, yi, city, size in zip(x, y, x.index, sizes):
            rs = city.split(' ')
            if len(rs) > 1:
                rsl = len(rs) // 2
                city = ' '.join(rs[:rsl] + ['\n'] + rs[rsl:])

            shar_path = [r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\Шар_{:02}.png".format(shar_num)]
            shar_img = OffsetImage(plt.imread(shar_path[0], format="png"), zoom=size)
            ab = AnnotationBbox(shar_img, (xi, yi), frameon=False)
            ax.add_artist(ab)
            plt.annotate(city, (xi, yi), ha='center', va='center', font=fpath, color='#404040')

        ax.hlines((y.min()+y.max())/2, x.min(), x.max(), colors='gray', linewidth=1)
        ax.vlines((x.min()+x.max())/2, y.min(), y.max(), colors='gray', linewidth=1)
        
        im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_шт_232_354_gray.png")
        newax = fig.add_axes([0.1, 0.8, 0.1, 0.1], zorder=1, alpha=0.5)
        newax.imshow(im)
        newax.axis('off')
        im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_2гис_232_354_gray.png")
        newax = fig.add_axes([0.9, 0.1, 0.1, 0.1], zorder=1, alpha=0.5)
        newax.imshow(im)
        newax.axis('off')
        fig.savefig(f'pictures/{folder}_shar_{shar_num}.png', bbox_inches='tight', transparent=False)
        
        txt = f''
        for cat in ['Звёзды:','Коровы:','Дети:','Собаки:']:
            txt +=  cat + ' ' + ', '.join([word[::-1].replace('/', 'и', 1)[::-1].replace('/', ',').replace(' ,', ',') for word in df[df['group'] == cat].index]) + "\n"
        txt = txt[::-1].replace('\n',':', 1)[::-1]
        add_image(f'files/{folder}/{folder}_shars_{time}.docx', f'pictures/{folder}_shar_{shar_num}.png', word=txt, img_w=img_w, img_h=img_h, blank=blank,word_sample=word_sample)
        if show:
            plt.show()
        plt.close()
        df_list.append(df)
    return pd.concat(df_list)

In [73]:
df = plot_matrix_rubric_cities(cnt_graphics=50, size_x=6.3*4, size_y=3.94*4, font_size=28, img_w=16.5, img_h=10, blank=True, word_sample='files/report_ten_city.docx', show=False)

files/ten_city/ten_city_shars_230503_2057.docx
[########################################] | 100% Completed | 1.81 sms


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [08:11<00:00,  9.83s/it]


In [76]:
df

,rubricName,city,Запросы 2020,Запросы 2021,Запросы 2022,Запросы 2023,"Запросы, итого","Рост/Падение, %","Доля рынка, %",group
city_rubric,,,,,,,,,,
Алматы.Торгово-развлекательные центры / Моллы,Торгово-развлекательные центры / Моллы,Алматы,1894280,1670773,2096427,3013600,300.000000,1.437493,1.563376e-02,Звёзды:
Алматы.Рестораны,Рестораны,Алматы,1497767,1601567,1820989,2954398,233.986274,1.622414,1.532664e-02,Звёзды:
Алматы.Быстрое питание,Быстрое питание,Алматы,1413635,1364990,1472664,2235546,119.513235,1.518029,1.159742e-02,Звёзды:
Алматы.Гостиницы,Гостиницы,Алматы,1091872,1096492,1334051,2199831,56.449824,1.648986,1.141214e-02,Звёзды:
Алматы.Бизнес-центры,Бизнес-центры,Алматы,1401014,1241767,1472430,1981680,87.350598,1.345857,1.028043e-02,Звёзды:
Астана.Кафе,Кафе,Астана,813791,1039594,1235852,1948604,0.000000,1.576729,1.010884e-02,Звёзды:
СПБ.Рестораны,Рестораны,СПБ,1389652,1246520,1573407,1735276,74.810646,1.102878,9.002155e-03,Собаки:
Москва.Рестораны,Рестораны,Москва,1599622,1351690,1801633,1716855,118.108186,0.952944,8.906592e-03,Собаки:
Москва.Кафе,Кафе,Москва,1405875,1063809,1345260,1346650,10.207165,1.001033,6.986066e-03,Собаки:
